In [3]:
from langchain_community.document_loaders.pdf import PyPDFLoader

arquivo = "./files/comunicando.pdf"

loader = PyPDFLoader(arquivo)
documento = loader.load()

In [4]:
len(documento)

3

In [5]:
documento[0]

Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2025-07-10T11:05:46-07:00', 'author': 'Lucas Abner', 'moddate': '2025-07-10T11:05:46-07:00', 'source': './files/comunicando.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='CONSUMINDO LLMs DENTRO DO MARVIN \n \nVamos descrever nessa documentação como enviar arquivos/containers para outra máquina e a \nfazer túnel entre portas. \n \n1.  Enviando Container para o Marvin \nAnteriormente, criamos um container singularity, onde está configurado para uso de GPUs \ndedicados ao Ollama. Então vamos passar esse arquivo para nosso HPC para que ele rode o Ollama \ne requisite direto do Servidor. Para mandar arquivos de sua máquina para outra é bem simples: \nscp ollama.sif <seu_usuario>@<servidor_hpc>:<caminho_para_deixar_arquivo> \nO que estamos fazendo aqui? Estamos transferindo uma cópia da sua máquina para o hpc, onde: \n• scp = é a cópia que você está fazendo \n• Ollama.sif = nome do arq

In [6]:
documento[2].metadata

{'producer': 'PyPDF',
 'creator': 'Microsoft Word',
 'creationdate': '2025-07-10T11:05:46-07:00',
 'author': 'Lucas Abner',
 'moddate': '2025-07-10T11:05:46-07:00',
 'source': './files/comunicando.pdf',
 'total_pages': 3,
 'page': 2,
 'page_label': '3'}

In [7]:
from langchain.chains.question_answering import load_qa_chain
from langchain_ollama.chat_models import ChatOllama

chat = ChatOllama(model="mistral:latest")

chain = load_qa_chain(llm=chat, chain_type="stuff", verbose=True)

/tmp/ipykernel_39470/1732760218.py:6: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm=chat, chain_type="stuff", verbose=True)


In [24]:
pergunta = "Tem algo falando sobre Ollama?"
chain.run(input_documents=documento[:2], question=pergunta)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
CONSUMINDO LLMs DENTRO DO MARVIN 
 
Vamos descrever nessa documentação como enviar arquivos/containers para outra máquina e a 
fazer túnel entre portas. 
 
1.  Enviando Container para o Marvin 
Anteriormente, criamos um container singularity, onde está configurado para uso de GPUs 
dedicados ao Ollama. Então vamos passar esse arquivo para nosso HPC para que ele rode o Ollama 
e requisite direto do Servidor. Para mandar arquivos de sua máquina para outra é bem simples: 
scp ollama.sif <seu_usuario>@<servidor_hpc>:<caminho_para_deixar_arquivo> 
O que estamos fazendo aqui? Estamos transferindo uma cópia da sua máquina para o hpc, onde: 
• scp = é a cópia que você está fazendo 
• Ollama.sif = nome do arq

' This document describes how to send files or containers to another machine and create a tunnel between ports. Here\'s the summary:\n\n1. Sending Container to Marvin: You can transfer an Singularity container file from your local machine to the HPC (Marvin) using the command `scp ollama.sif <your_username>@<server_hpc>:<path_to_save_file>`. This command copies the specified file from your machine to the HPC.\n\n2. Tunnel Between Ports Ollama on Marvin: After sending the container, you\'ll create a connection tunnel between the port ollama on your local machine (port 11434) and the port ollama of the container running on Marvin using the command `ssh –L 11434:localhost:11434 <your_username>@<server_hpc>`. This command sets up a secure connection between your local machine and the HPC.\n\n3. Linking Ollama and Requisition via Tunnel: With the container on the remote machine and the necessary tunnel created, you need to activate the container to use your LLM in the project. Use the comma

## Carregando videos do Youtube

In [1]:
from langchain_core.document_loaders import BaseLoader
from langchain_community.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
import whisper
from langchain_core.documents import Document

In [2]:
url = "https://www.youtube.com/watch?v=Tm9y9pM5dJc"
save_dir = "files/youtube"

def transcribe_audio_local(audio_path):
        model = whisper.load_model("base")
        result = model.transcribe(audio_path)
        return Document(page_content=result["text"])

class LocalWhisperLoader(BaseLoader):
    def __init__(self, urls, save_dir):
        self.urls = urls
        self.save_dir = save_dir

    def load(self):
        loader = YoutubeAudioLoader(self.urls, self.save_dir)
        docs = []
        for blob in loader.yield_blobs():
            audio_path = blob.path
            doc = transcribe_audio_local(str(audio_path))
            docs.append(doc)
        return docs

url = "https://www.youtube.com/watch?v=00Q0G84kq3M"
loader = LocalWhisperLoader([url], "files/youtube")
docs = loader.load()

[youtube] Extracting URL: https://www.youtube.com/watch?v=00Q0G84kq3M
[youtube] 00Q0G84kq3M: Downloading webpage
[youtube] 00Q0G84kq3M: Downloading tv client config
[youtube] 00Q0G84kq3M: Downloading tv player API JSON
[youtube] 00Q0G84kq3M: Downloading ios player API JSON
[youtube] 00Q0G84kq3M: Downloading m3u8 information
[info] 00Q0G84kq3M: Downloading 1 format(s): 140
[download] files/youtube/RAG vs. Fine Tuning.m4a has already been downloaded
[download] 100% of    8.28MiB
[ExtractAudio] Not converting audio files/youtube/RAG vs. Fine Tuning.m4a; file is already in target format m4a


In [8]:
pergunta = "Faça um resumo desse video"
chain.run(input_documents = docs, question = pergunta)

/tmp/ipykernel_39470/1681284590.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents = docs, question = pergunta)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
 In this course, Lance Martin will teach you how to implement RAG from scratch. Lance is a software engineer at Langchain. And Langchain is one of the most common ways to implement RAG. Lance will help you understand how to use RAG to combine custom data with LLMs. Hi, this is Lance Martin. I'm a software engineer at Langchain. And we're giving a short course, focused on RAG or a tree vlogmented generation, which is one of the most popular kind of ideas and applications in LLMs today. So really the motivation for this is that most of the world's data is private. Whereas LLMs are trained on publicly available data. So you can kind of see on the bottom on the x-axis, the number of tokens using pre-trai

' O vídeo discute sobre como as pessoas podem adotar uma abordagem mais positiva e saudável para o uso de tecnologia. Ele enfatiza a importância da consciência digital, onde as pessoas se tornam mais conscientes do impacto que o uso da tecnologia tem em sua vida, incluindo a maneira como ela pode afetar a saúde mental e emocional.\n\nO vídeo também destaca alguns dos riscos associados ao uso excessivo de tecnologia, como ansiedade, depressão e baixo desempenho acadêmico. Além disso, ele oferece algumas dicas práticas para adotar uma abordagem mais sintonizada com a vida real, incluindo definir limites para o uso da tecnologia, usando técnicas de organização para evitar distração e tomando pequenos pauses ao longo do dia para se desconectar e relaxar.\n\nO vídeo também apresenta alguns benefícios potenciais da tecnologia, incluindo a capacidade de conectar-se com pessoas em todo o mundo e ter acesso a uma grande quantidade de informação. Por fim, ele incentiva as pessoas a encontrarem u

## Web via URL

In [9]:
from langchain_community.document_loaders.web_base import WebBaseLoader

url = "https://python.langchain.com/docs/tutorials/rag/"
loadd = WebBaseLoader(url)

docu = loadd.load()

In [10]:
len(docu)

1

In [11]:
docu[0].page_content[:500]

'\n\n\n\n\nBuild a Retrieval Augmented Generation (RAG) App: Part 1 | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuil'

In [12]:
perguntaa = "O que é SearchIntroductionTutorialsBuild?"

chain.run(input_documents = docu, question = perguntaa)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------





Build a Retrieval Augmented Generation (RAG) App: Part 1 | 🦜️🔗 LangChain






Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answerin

' "Search Introduction Tutorials Build" não parece ser uma expressão comum ou um termo específico em inglês. É possível que esteja faltando alguma informação importante para entender corretamente o significado.\n\nSe estiver se referindo a uma página de introdução às aulas de aprendizagem sobre como usar um tutorial de busca, poderia ser traduzida como "Introdução das Aulas de Tutorial de Busca". Se for um recurso específico em algum sistema de busca ou plataforma de aprendizado, é possível que esteja faltando a informação necessária para identificar o contexto.\n\nSe alguém puder fornecer mais informações sobre onde e em qual contexto esta expressão está sendo usada, seria possível fornecer uma tradução mais precisa e apropriada.'